In [176]:
import glob
import re
import ast
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd

# 1. Paragraph에서 숫자 추출

In [166]:
all_path = glob.glob('paragraph/income_quantile*/PL_*.csv')

In [168]:
# 1. paragraph에서 숫자 추출
col = 'desc'

def extract_numbers_after_country(text):
    # 국가 코드 중 하나 찾기
    match = re.search(r"(POL|SVK|CZE|HUN)", text)
    if not match:
        return []

    # 그 위치 이후의 문자열 자르기
    start_idx = match.end()
    tail = text[start_idx:]

    # 숫자 추출 (정수, 소수, 음수 포함)
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", tail)
    return [float(num) for num in numbers]

for path in all_path:
    df = pd.read_csv(path)
    df["numbers"] = df[col].astype(str).apply(extract_numbers_after_country)
    df.to_csv(path)

# 2-1. feature normalization (새로운 feature인 경우)

In [197]:
# 2-1. feature normalization (새로운 feature가 주어졌을 경우)

score = {
    'Very Low': 0,
    'Low': 1,
    'Medium': 2,
    'High': 3,
    'Very High': 4,
}

isshot = False

all_result = glob.glob('class/income_quantile*/*/*.csv')


for path in all_result:
    parameter = path.split('/')[1]
    
    
    
    if isshot:
        paragraph = pd.read_csv(f'./paragraph/{parameter}/PL_SK_CZ_HU_{parameter}_adm2_9549_o1mini_landuse_wo_neighbor_paragraph_output.csv')
    else:
        paragraph = pd.read_csv(f'./paragraph/{parameter}/PL_{parameter}_adm2_9549_paragraph_output.csv')
    
    # 문자열 -> 리스트 변환
    paragraph['numbers'] = paragraph['numbers'].apply(ast.literal_eval)
    vector_df = paragraph['numbers'].apply(pd.Series)
    # 모든 값이 같은 컬럼만 골라냄
    constant_columns = vector_df.columns[vector_df.nunique() == 1]
    
    # 제거
    vector_df = vector_df.drop(columns=constant_columns)

    vector_df.columns = [f"feature_{i}" for i in range(vector_df.shape[1])]

    # feature 정규화
    scaler = MinMaxScaler()
    scaled_array = scaler.fit_transform(vector_df)
    normalized_df = pd.DataFrame(scaled_array, columns=vector_df.columns)

    df_expanded = pd.concat([paragraph[["area_id", 'desc']], normalized_df], axis=1)

    df = pd.read_csv(path)
    try:
        df['cluster'] = df['score'].apply(lambda s: score[s])
    except:
        print(path)
        continue
    df = pd.merge(df, df_expanded, how='inner', on='area_id')
        
    df = df.drop(columns=['score', 'ground_truth', 'desc'])
    filename = path.split('/')[-1].split('.')[0]
    df.to_csv(f'./data/income_quantile/{filename}.csv', index=False)

# 2-2. 이미 feature extraction 진행한 경우, 원래 값 활용

In [182]:
score = {
    'Very Low': 0,
    'Low': 1,
    'Medium': 2,
    'High': 3,
    'Very High': 4,
}

new_class = glob.glob(f'./class/income_quantile*/*/*.csv')

name = 'income_quantile'
model = 'o4-mini'

os.makedirs(f'./data/{name}', exist_ok=True)
for csv in new_class:
    df = pd.read_csv(csv)
    try: 
        df['cluster_new'] = df['score'].apply(lambda x: score[x])
    except:
        print(csv)
        continue
    filename = csv.split('/')[-1]
    o1filename = filename.replace(model, 'o1-mini').replace('_addr', '').replace('_random', '')
    feature = pd.read_csv(f'./data/o1mini/{o1filename}')
    df = df.merge(feature, on='area_id', how='inner')
    df['cluster'] = df['cluster_new']
    df = df.drop(columns=['ground_truth', 'score', 'cluster_new'])
    if not os.path.exists(f'./data/{name}/{filename}'):
        df.to_csv(f'./data/{name}/{filename}', index=False)

income_quantile4


NameError: name 'PL_income_quantile1_adm2_9549_paragraph_output' is not defined

In [119]:
score = {
    'Very Low': 0,
    'Low': 1,
    'Medium': 2,
    'High': 3,
    'Very High': 4,
}

# os.makedirs(f'./data/{name}')
for csv in new_class:
    df = pd.read_csv(csv)
    try: 
        df['cluster_new'] = df['score'].apply(lambda x: score[x])
    except:
        print(csv)
        continue
    filename = csv.split('/')[-1]
    feature = pd.read_csv(f'./data/o1mini/{filename}')
    print(df)
    df = df.merge(feature, on='area_id', how='inner')
    df['cluster'] = df['cluster_new']
    print(df)
    df = df.drop(columns=['ground_truth', 'score', 'cluster_new'])
    if not os.path.exists(f'./data/shot/{filename}'):
        df.to_csv(f'./data/shot/{filename}', index=False)

  area_id      score  sample_val  ground_val  cluster_new
0   HU313   Very Low       424.0         424            0
1   PL812        Low       737.6         603            1
2   PL714        Low       737.6         648            1
3   PL218        Low       737.6         678            1
4   PL912        Low       737.6         690            1
5   PL411        Low       737.6        1069            1
6   PL414     Medium      1328.5        1327            2
7   PL517     Medium      1328.5        1330            2
8   CZ064       High      4336.0        4336            3
9   CZ010  Very High      6979.0        6979            4
Empty DataFrame
Columns: [area_id, score, sample_val, ground_val, cluster_new, cluster, feature_0, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6]
Index: []


KeyError: "['ground_truth'] not found in axis"

# 3. 결과 중 총 class 개수가 5 미만인 것 확인(다른 POG 사용해야 함.)

In [198]:
# 3. 결과 중 총 class 개수가 5 미만인 것 확인(다른 POG 사용해야 함.)

all_result = glob.glob(f'./data/income_quantile/*.csv')

for path in all_result:
    df = pd.read_csv(path)
    if len(df['cluster'].unique()) != 5:
                
        # 고유값 정렬해서 매핑 딕셔너리 만들기
        unique_sorted = sorted(df['cluster'].unique())
        mapping = {old: new for new, old in enumerate(unique_sorted)}

        # 매핑 적용
        df['cluster'] = df['cluster'].map(mapping)
        df.to_csv(path, index=False)
        print(path, unique_sorted)

./data/income_quantile/PL_income_quantile4_classification_o4-mini_10shots_iter_num_1_class_sample_6.csv [1, 2, 3, 4]
./data/income_quantile/PL_income_quantile1_classification_o4-mini_10shots_iter_num_2_class_sample_8.csv [0, 1, 2, 3]
